<a href="https://colab.research.google.com/github/savinyu/VLG_Project/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.utils import compute_class_weight


train_df = pd.read_csv('train.csv')

X = train_df.drop('labels', axis=1).values
Y = train_df['labels'].values


X = X.reshape(X.shape[0], X.shape[1], 1)
X = X / 4



X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))



# Model

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(1200, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=64, kernel_size=3, strides=2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.4))

model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.45))

model.add(Dense(1, activation='sigmoid'))

# tf.keras.applications.ResNet50(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,

# )
# tf.keras.applications.VGG19(
#     include_top=True,
#     weights="imagenet",
#     input_tensor=None,
#     input_shape=None,
#     pooling=None,
#     classes=1000,
#     classifier_activation="softmax",
# )



model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=128, epochs=10, class_weight=class_weights_dict)

y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)
f1 = f1_score(y_test, y_pred)
print("F1 score:", f1)


test_df = pd.read_csv('test.csv')


test = test_df.drop('id', axis=1).values
test = test.reshape(test.shape[0], test.shape[1], 1)


pred_probs = model.predict(test)
predicted_labels = (pred_probs > 0.5).astype(int)
predicted_labels = predicted_labels.reshape(predicted_labels.shape[0])

import csv

output_data = []
for i, value in enumerate(predicted_labels):
    output_data.append([i + 1, value])

output_file = 'output.csv'

with open(output_file, 'w', newline='') as file:
    writer = csv.writer(file)

    writer.writerow(['id', 'label'])
    writer.writerows(output_data)



print("CSV Output Done")


from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

Epoch 1/10
27/27 [==============================] - 3s 87ms/step - loss: 0.6788 - accuracy: 0.5576
Epoch 2/10
27/27 [==============================] - 3s 111ms/step - loss: 0.5757 - accuracy: 0.6507
Epoch 3/10
27/27 [==============================] - 3s 92ms/step - loss: 0.5036 - accuracy: 0.7303
Epoch 4/10
27/27 [==============================] - 2s 87ms/step - loss: 0.4226 - accuracy: 0.7852
Epoch 5/10
27/27 [==============================] - 2s 87ms/step - loss: 0.3638 - accuracy: 0.8419
Epoch 6/10
27/27 [==============================] - 2s 86ms/step - loss: 0.3378 - accuracy: 0.8484
Epoch 7/10
27/27 [==============================] - 3s 119ms/step - loss: 0.2731 - accuracy: 0.8899
Epoch 8/10
27/27 [==============================] - 2s 87ms/step - loss: 0.2363 - accuracy: 0.8998
Epoch 9/10
27/27 [==============================] - 2s 86ms/step - loss: 0.1906 - accuracy: 0.9275
Epoch 10/10
27/27 [==============================] - 0s 7ms/step
F1 score: 0.6276595744680851
71/71 [======